## SP III-1 Prüfungsleistung - Einführung

Es liegen Daten aus dem Zensus 2022 vor, die für Analysen vorbereitet werden sollen.
Spezifisch soll die Auswertbarkeit nach Ländern und Regionalebene ermöglicht werden. Im Folgenden wird der ETL-Prozess dargestellt mit dem Zensusdaten aus einer Excel-Datei extrahiert, bereinigt und in ein Sternschema für eine PostgreSQL-Datenbank überführt werden. Der Fokus liegt darauf, eine Datenstruktur zu entwickeln, die schnelle und effiziente analytische Abfragen ermöglicht.
Erste visuelle Analysen werden als Abschluss präsentiert.

## 1. Dateninfastruktur aufsetzen

1. PostgreSQL Server lokal aufsetzen
- Zunächst erfolgt Authentifizierung und Herstellung einer Datenbanksitzung in PostgreSQL über die psql-Konsole.
- PostgreSQL-Datenbank und Benutzer konfigurieren

2. VS Code und Python
3. pgAdmin für die Verwaltung

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, Float, MetaData, ForeignKey, text
import numpy as np
from psycopg2 import sql
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import locale
from IPython.display import display

## 2. Überführung der Daten in eine relationale Datenbank 

## A. Extract
Extrahieren von 10 Tabellenblättern aus einer Excel-Datei, die in der Benennung mit "CSV-" beginnen.

In [ ]:
# Basispfade und Dateinamen
base_path = '/Users/sabinewildemann/Desktop/Studium/Pruefungsleistung/'
input_path = os.path.join(base_path, 'input')
output_path = os.path.join(base_path, 'output')
file_name = "Regionaltabelle_Bildung_Erwerbstaetigkeit 1.xlsx"
file_path = os.path.join(input_path, file_name)

# Einlesen der Excel-Datei
xls = pd.ExcelFile(file_path)

# Einlesen und Laden der Attribute Details
attribute_details_df = xls.parse('Attribute_Details')
attribute_details_df['attribut_id'] = range(1, len(attribute_details_df) + 1)

# Attribute Details in einem Dictionary speichern für schnellen Zugriff
attribute_details_dict = dict(zip(attribute_details_df['Attribut'], attribute_details_df['attribut_id']))

# Datenframes für die Daten und Attribute vorbereiten
data_frames = {}

# Festlegen, welche Sheets ausgelesen werden sollen
sheets_to_read = [sheet for sheet in xls.sheet_names if sheet.startswith("CSV-")]

# Daten für jede Entität einlesen und verarbeiten
for sheet_name in sheets_to_read:
    df = xls.parse(sheet_name, dtype={'_RS': str})
    clean_name = sheet_name.replace("CSV-", "")
    
    # Definieren der Muster, die für Summenspalten stehen und daher ausgeschlossen werden sollen
    summen_spalten_muster = ('_STP', '_STP__M', '_STP__W', '_ERW', '_ERW__M', '_ERW__W')

    # Anpassen des DataFrame, indem alle Spalten, die auf die definierten Muster enden, ausgeschlossen werden
    df = df[[col for col in df.columns if not col.endswith(summen_spalten_muster)]]
    
    # Hinzufügen von Attributdetails und Entität-Name
    df['Entität'] = clean_name
    for col in df.columns:
        if col in attribute_details_dict:
            df[col + '_detail'] = attribute_details_dict[col]

    data_frames[clean_name] = df
    
# Anzahl der geladenen Tabellen und deren Namen ausgeben
print(f"Anzahl der geladenen Tabellen: {len(sheets_to_read)}")
print(f"Geladene Tabellen und entsprechende DataFrame-Namen: {list(data_frames.keys())}")

# Überprüfung der Daten
for sheet, df in data_frames.items():
    print(f"\nDatenüberprüfung für {sheet}:")
    display(df.head(10))    # YY so kann man das df schöner darstellen, beachte dazu den import von display weiter oben


### Cleansing und Profiling report

In [ ]:
#Lt. Erläuterungen zur Methodik in der Zensus 2022_Datei gibt es einige Sonderzeichen, die zu berücksichtigen sind:
#–= Genau Null oder auf Null geändert
#/= Keine Angabe, da Zahlenwert nicht sicher genug
#.= Zahlenwert unbekannt oder geheim

def replace_special_characters(val): 
    if isinstance(val, str):
        # Entferne führende und nachfolgende Leerzeichen
        val = val.strip()
        
        # Prüfe, ob der Wert ein einzelner langer Gedankenstrich ist
        if val == '–':
            return 0
        elif val == '/' or val == '.':
            return np.nan
    
    # Gibt den ursprünglichen Wert zurück, wenn keine Ersetzung erforderlich ist
    return (val)

def count_and_replace_special_characters(df):
    special_chars = ['–', '.', '/']  # Verwende nur die relevanten Sonderzeichen
    counts = {char: 0 for char in special_chars}
    
    for col in df.columns:
        if df[col].dtype == object:
            # Zählen der Sonderzeichen
            for char in special_chars:
                counts[char] += df[col].apply(lambda x: str(x).strip() == char).sum()
            
            # Ersetzen der Sonderzeichen nur, wenn sie allein in einer Zelle stehen
            # YY das aus dem Kommentar wird hier aber nicht gemacht
            df[col] = df[col].apply(replace_special_characters)
        else:
            # Wenn die Spalte numerisch ist, gibt es keine Sonderzeichen
            continue
    
    return counts

# Überprüfe jedes Sheet, zähle die Sonderzeichen und ersetze sie
for sheet, df in data_frames.items():
    special_char_counts = count_and_replace_special_characters(df)
    print(f"Gefundene und ersetzte spezielle Zeichen im DataFrame {sheet}:")
    for char, count in special_char_counts.items():
        print(f"Anzahl von '{char}': {count}")

# Optional: Überprüfung der Ergebnisse
for sheet_name, df in data_frames.items():
    print(f"\nÜberprüfung des DataFrames '{sheet_name}':")
    display(df.head(10))


In [ ]:
# YY unnötig?

def check_for_special_characters(df):
    # Definiere die Sonderzeichen, die überprüft werden sollen
    special_chars = ['–', '.', '/']
    # Überprüfe nur die ersten fünf Spalten
    for col in df.columns[:5]:
        if df[col].dtype == object:
            for char in special_chars:
                if df[col].astype(str).apply(lambda x: x == char).any():
                    return True
    return False

# Überprüfe alle DataFrames und gib aus, ob noch Sonderzeichen in den ersten fünf Spalten vorhanden sind
for sheet_name, df in data_frames.items():
    if check_for_special_characters(df):
        print(f"Sonderzeichen gefunden im DataFrame '{sheet_name}'!")
    else:
        print(f"Keine Sonderzeichen gefunden im DataFrame '{sheet_name}'.")

In [ ]:
# Übersichtstabelle erstellen und Daten prüfen
summary_data = []
for sheet, df in data_frames.items():
    # Reihenfolge der Spalten anpassen, um "Entität" als fünfte Spalte zu positionieren
    # Die Spalte "Entität" sollte vor den numerischen Spalten stehen
    column_order = ['_RS', 'Name', 'Regionalebene', 'Berichtszeitpunkt', 'Entität'] + [col for col in df.columns if col not in ['_RS', 'Name', 'Regionalebene', 'Berichtszeitpunkt', 'Entität']]
    df = df[column_order]

    # Spalten ab der sechsten Spalte in numerische Werte konvertieren, da "Entität" jetzt die fünfte Spalte ist
    for col in df.columns[5:]:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

    # Nummer der Spalten und ihre Datentypen speichern
    num_columns = df.shape[1]
    dtypes = df.dtypes
    dtypes_str = ", ".join([f"{col}: {dtype}" for col, dtype in dtypes.items()])

    # Daten für die Zusammenfassungstabelle sammeln
    summary_data.append([sheet, num_columns, dtypes_str])
    print(f"Sheet: {sheet}")
    display(df.head())

# Zusammenfassungstabelle erstellen und ausgeben
summary_df = pd.DataFrame(summary_data, columns=["Sheet Name", "Number of Columns", "Column Data Types"])
display(summary_df)

# #Sweetviz-Berichte generieren
# for sheet, df in data_frames.items():
#     try:
#         print(f"Erzeuge Sweetviz-Bericht für das Tabellenblatt: {sheet}")
#         report = sv.analyze(df)
#         report_file_path = os.path.join(output_path, f"sweetviz_report_{sheet}.html")
#         report.show_html(report_file_path)
#         print(f"Sweetviz-Bericht gespeichert: {report_file_path}")
#     except Exception as e:
#         print(f"Fehler bei der Erstellung des Sweetviz-Berichts für {sheet}: {e}")

# print("Sweetviz-Berichte wurden als HTML-Dateien gespeichert.")

## 2. Transform
Im Datenbereinigungs-Schritt werden folgende Anpassungen in den Spalten vorgenommen:
- "Name" in Data type "String" konvertiert
- "Regionalebene" in Data type "String" konvertiert und überschüssige Leerzeichen entfernt
- "Entität" in Data type "String" konvertiert  
- Alle Spaltenbezeichnungen in Kleinbuchstaben ändern

In [ ]:
def transform_data(data_frames):
    for sheet, df in data_frames.items():
        print(f"\nVerarbeite Sheet: {sheet}")
        
        # Konvertiere 'Name' und 'Regionalebene' zu Strings und normalisiere sie
        if 'Name' in df.columns:
            df['Name'] = df['Name'].astype(str)
            print("Name konvertiert zu String.")
        
        if 'Regionalebene' in df.columns:
            df['Regionalebene'] = df['Regionalebene'].astype(str)
            df['Regionalebene'] = df['Regionalebene'].str.strip()
            print("Regionalebene konvertiert zu String und normalisiert.")

        if 'Entität' in df.columns:
            df['Entität'] = df['Entität'].astype(str)
            print("Entität konvertiert zu String.")

        # Ändere alle Spaltennamen in Kleinbuchstaben
        df.columns = [col.lower() for col in df.columns]
        print(f"Spaltennamen in '{sheet}' zu Kleinbuchstaben geändert.")

        # Speichere den veränderten DataFrame zurück in das Dictionary
        data_frames[sheet] = df

    print("\nTransformation abgeschlossen.")
    
    # Rückgabe des aktualisierten Dictionaries zur weiteren Verwendung oder Überprüfung
    return data_frames

# Anwendung der Transformation und Speichern des Ergebnisses
transformed_data_frames = transform_data(data_frames)

# Drucke die Spaltennamen für jede Tabelle zur Überprüfung
for sheet, df in transformed_data_frames.items():
    print(f"Spaltennamen in '{sheet}': {df.columns.tolist()}")


In [ ]:
df.head()

In [ ]:
#Überprüfung der Summendaten der Regionalebenen
def verify_sum_data(data_frames):
    # Liste der Regionalebenen
    region_levels = ['Gemeinde', 'Gemeindeverband', 'Stadtkreis/kreisfreie Stadt/Landkreis', 'Regierungsbezirk', 'Land', 'Bund']

    # Berechnung der Summen für jede Regionalebene
    for sheet, df in data_frames.items():
        print(f"\nBerechnungen für Sheet: {sheet}")
        for level in region_levels:
            if 'regionalebene' in df.columns:
                # Der Index für die Spalte direkt rechts von 'Regionalebene'
                region_index = df.columns.get_loc('regionalebene')
                sum_column_name = df.columns[region_index + 1]  # Nimmt die Spalte direkt nach 'Regionalebene'
                
                # Filtern der Daten nach der Regionalebene und Überprüfen der Summen
                filtered_df = df[df['regionalebene'] == level]
                if not filtered_df.empty:
                    level_sum = filtered_df[sum_column_name].sum()
                    print(f"Gesamtsumme für {level} in {sheet}: {level_sum}")
                else:
                    print(f"Keine Daten gefunden für {level} in {sheet}")

verify_sum_data(data_frames)


In [ ]:
# Überprüfung der Verteilung der Regionalebenen in allen DataFrames
region_level_distribution = {}
for sheet, df in data_frames.items():
    distribution = df['regionalebene'].value_counts(normalize=True, sort=False).sort_index()
    region_level_distribution[sheet] = distribution

# Vergleichen der Verteilungen
base_distribution = None
consistent = True
for sheet, dist in region_level_distribution.items():
    if base_distribution is None:
        base_distribution = dist
    elif not dist.equals(base_distribution):
        consistent = False
        break

if consistent:
    print("Die Verteilung der Regionalebenen ist über alle DataFrames hinweg konsistent.")
else:
    print("Die Verteilung der Regionalebenen ist nicht konsistent.")

# Ausgabe der Verteilung für alle DataFrames
for sheet, dist in region_level_distribution.items():
    print(f"Verteilung für {sheet}:\n{dist}\n")

In [ ]:
# Analyse der Struktur des Regionalschlüssels '_RS' im Zusammenhang mit der 'Regionalebene'
rs_structure = {}
for sheet, df in data_frames.items():
    rs_length = df.apply(lambda row: (row['regionalebene'], len(str(row['_rs']))), axis=1)
    rs_structure[sheet] = rs_length.value_counts()

# Überprüfung der Konsistenz der Struktur
base_structure = None
structure_consistent = True
for sheet, structure in rs_structure.items():
    if base_structure is None:
        base_structure = structure
    elif not structure.equals(base_structure):
        structure_consistent = False
        break

if structure_consistent:
    print("Die Struktur des Regionalschlüssels ist über alle DataFrames hinweg konsistent.")
else:
    print("Die Struktur des Regionalschlüssels ist nicht konsistent.")

# Ausgabe der Strukturen für alle DataFrames
for sheet, structure in rs_structure.items():
    print(f"Struktur für {sheet}:\n{structure}\n")

In [ ]:
#Regionalebenen 'Gemeinde', 'Gemeindeverband', 'Regierungsbezirk', 'Bund' entfernen, um Redundanz und unvollständiefe Daten zu vermeiden
def remove_specific_region_levels(data_frames):
    # Liste der zu entfernenden Regionalebenen
    levels_to_remove = ['Gemeinde', 'Gemeindeverband', 'Regierungsbezirk', 'Bund']

    for sheet_name, df in data_frames.items():
        if 'regionalebene' in df.columns:
            # Entferne die Zeilen, die die spezifischen Regionalebenen enthalten
            df = df[~df['regionalebene'].isin(levels_to_remove)]
            data_frames[sheet_name] = df  # Aktualisiere den DataFrame im Dictionary
            print(f"Die Regionalebenen {levels_to_remove} wurden aus dem DataFrame '{sheet_name}' entfernt.")

# Wende die Funktion auf die DataFrames an
remove_specific_region_levels(data_frames)

# Optional: Überprüfung der Ergebnisse
for sheet_name, df in data_frames.items():
    print(f"\nÜberprüfung des DataFrames '{sheet_name}':")
    print(df['regionalebene'].value_counts())

## C. Load

### 3. Überführung in ein Sternschema
### 3a: Design eines Sternschemas mit Dimensionstabellen und einer Faktentabelle
### 3b: Überführen der Daten in das Sternschema - Entpivotieren

Nach dem Entpivotieren werden die Dimensionen in separate Tabellen extrahiert und durch referenzielle IDs in der Faktentabelle ersetzt. Dies wird durch die Erstellung eines eindeutigen Identifikators für jede Dimension erreicht und dann die Zuordnung dieser IDs in der Faktentabelle.
Die Daten werden von einem breiten zu einem langen Format transformiert, wobei jede Kennzahl in einer eigenen Zeile dargestellt wird.

In [ ]:
# Zugangsdaten für PostgreSQL Server und Datenbank
host = "localhost"
port = "5432"
database = "studienarbeit_dateninfrastruktur"
user = "postgres"
password = "5566"

# Verbindungsstring mit Datenbankdetails zusammenstellen
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
metadata = MetaData()

In [ ]:
# 'dimensions' Dictionary initialisieren
dimensions = {}

# Kombinieren aller DataFrame-Regioneninformationen
region_data = []
for df in data_frames.values():
    if {'_rs', 'name', 'regionalebene'}.issubset(df.columns):
        region_data.append(df[['_rs', 'name', 'regionalebene']].drop_duplicates().reset_index(drop=True))
        
region_data

In [ ]:
# Überführen in 2. Normalform und Erstellen der Dimensionstabellen im Metadata.
# Alle Nicht-Schlüsselattribute hängen vollständig vom Primärschlüssel ab.
# Dimensionstabellen definieren
dim_regionen = Table('dim_regionen', metadata,
    Column('region_id', String, primary_key=True),
    Column('name', String),
    Column('regionalebene', String)
)
# Erstellen der einheitlichen Dimensionstabelle für Regionen
combined_region_data = pd.concat(region_data).drop_duplicates().reset_index(drop=True)

combined_region_data.rename(columns={'_rs': 'region_id'}, inplace=True)  # Hier '_rs' in 'region_id' umbenennen
dimensions['dim_regionen'] = combined_region_data

In [ ]:
combined_region_data

In [ ]:
# Erstellen der Dimensionstabelle für Berichtszeitpunkte
all_dates = pd.concat([df['berichtszeitpunkt'] for df in data_frames.values() if 'berichtszeitpunkt' in df.columns], ignore_index=True).drop_duplicates()
# Convert to datetime and back to string to ensure valid dates and consistent formatting
all_dates = pd.to_datetime(all_dates, format='%Y%m%d').dt.strftime('%Y%m%d')
dimensions['dim_zeitpunkte'] = pd.DataFrame({
    'berichtszeitpunkt': all_dates,
    'time_id': range(1, len(all_dates) + 1)
})

# Erstellen der Dimensionstabelle für Geschlecht
geschlecht_data = pd.DataFrame({
    'geschlecht': ['personen_total', 'männlich', 'weiblich'],
    'geschlecht_id': [1, 2, 3]
})
dimensions['dim_geschlecht'] = geschlecht_data

# Erstellen der Dimensionstabelle für Entitäten
dimensions['dim_entitaeten'] = pd.DataFrame({
    'entitaet': list(data_frames.keys()),
    'entitaet_id': range(1, len(data_frames) + 1)
})

# Erstellen der Dimensionstabelle für Attributdetails
attribute_details_df['attribut_id'] = range(1, len(attribute_details_df) + 1)
attribute_details_df['attribut'] = attribute_details_df['Attribut'].str.lower()
# Cleaning special characters and spaces in the dictionary keys
attribute_details_df['attribut'] = attribute_details_df['attribut'].str.replace('–', '').str.strip()
attribute_details_df['detail'] = attribute_details_df['Detail'].str.lower()
dimensions['dim_attribute_details'] = attribute_details_df[['attribut_id', 'attribut', 'detail']]

# Create the dictionary for attribute ID mapping
attribute_details_dict = attribute_details_df.set_index('attribut')['attribut_id'].to_dict()

In [ ]:
# Explicitly create dimension tables in the SQL database, even if they are empty
for table_name, df in dimensions.items():
    print(f"Überprüfung der Tabelle {table_name}, DataFrame hat {len(df)} Zeilen.")
    
    # Wenn der DataFrame leer ist, sicherstellen, dass die Tabelle trotzdem erstellt wird
    if df.empty:
        # Erstellen des Tabellenschemas ohne Daten einzufügen
        pd.io.sql.get_schema(df, table_name, con=engine)
        print(f"Tabellenschema für {table_name} wurde erstellt, da es leer war.")
    else:
        # Daten in die entsprechende Tabelle laden
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)
        print(f"Dimensionstabelle {table_name} wurde erfolgreich hochgeladen.")

In [ ]:
faktendaten = []

for sheet, df in data_frames.items():
    df['entitaet_id'] = dimensions['dim_entitaeten'].set_index('entitaet').loc[sheet, 'entitaet_id']
    id_vars = ['_rs', 'name', 'regionalebene', 'entitaet_id']
    if 'berichtszeitpunkt' in df.columns:
        id_vars.append('berichtszeitpunkt')
        df['berichtszeitpunkt'] = df['berichtszeitpunkt'].astype(str)

    value_vars = [col for col in df.columns if col not in id_vars and not col.endswith('_detail')]

    # Entpivotieren
    df_long = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='kennzahlentyp', value_name='kennzahlenwert')

    if 'berichtszeitpunkt' in df_long.columns:
        df_long['berichtszeitpunkt'] = df_long['berichtszeitpunkt'].astype(str)
        df_long = df_long.merge(
            dimensions['dim_zeitpunkte'][['berichtszeitpunkt', 'time_id']],
            on='berichtszeitpunkt',
            how='left') 

    df_long['kennzahlentyp'] = df_long['kennzahlentyp'].str.strip().str.lower()
    df_long['attribut_id'] = df_long['kennzahlentyp'].map(attribute_details_dict).fillna(0).astype(int) 

    df_long = df_long.merge(dimensions['dim_attribute_details'][['attribut_id', 'detail']], on='attribut_id', how='left')

    # Bestimmung des Geschlechts basierend auf 'kennzahlentyp'
    df_long['geschlecht'] = df_long['kennzahlentyp'].apply(
        lambda x: 'personen_total' if '_M' not in x and '_W' not in x else ('männlich' if '_M' in x else 'weiblich')
    )
    df_long = df_long.merge(dimensions['dim_geschlecht'], on='geschlecht', how='left')

    if '_rs' in df_long.columns:
        df_long.rename(columns={'_rs': 'region_id'}, inplace=True)

    df_long = df_long.merge(dimensions['dim_regionen'], on=['region_id', 'name', 'regionalebene'], how='left')

    # Drop columns only if they exist in the DataFrame
    columns_to_drop = ['_rs', 'name', 'regionalebene', 'berichtszeitpunkt', 'geschlecht', 'attribut_id']
    df_long.drop(columns=[col for col in columns_to_drop if col in df_long.columns], inplace=True)

    # 'kennzahlenwert' zu numerischen Werten konvertieren und ungültige Einträge entfernen
    df_long['kennzahlenwert'] = pd.to_numeric(df_long['kennzahlenwert'], errors='coerce')
    df_long.dropna(subset=['kennzahlenwert'], inplace=True)
    df_long.columns = [col.lower() for col in df_long.columns]
        
    faktendaten.append(df_long)


In [ ]:
df_long

In [ ]:
# Zusammenführen aller Daten in einer einzigen Faktentabelle
fct_zensus = pd.concat(faktendaten, ignore_index=True)

# Definition und Erstellung der Faktentabelle in der Datenbank
fct_zensus_table = Table('fct_zensus', metadata,
    Column('index', Integer, primary_key=True),
    Column('kennzahlentyp', String),
    Column('detail', String),
    Column('kennzahlenwert', Integer),
    Column('region_id', String),
    Column('time_id', Integer),
    Column('entitaet_id', Integer),
    Column('geschlecht_id', Integer),
    Column('attribut_id', Integer),
    extend_existing=True
)

In [ ]:
# Erstellen der Tabellen und Laden der Daten in die Datenbank
metadata.create_all(engine)

In [ ]:
# Load fact table data
fct_zensus.to_sql('fct_zensus', engine, if_exists='replace', index=False)

# Prüfung der finalen Struktur der Faktentabelle in der Datenbank
query_columns = "SELECT * FROM fct_zensus LIMIT 0;"
df_columns = pd.read_sql(query_columns, engine)
print("Spalten in fct_zensus nach Update:")
print(df_columns.columns.tolist())

print("Alle Daten wurden erfolgreich in die Datenbank geladen.")

## 3d: Visualisierung der Daten

In [ ]:
locale.setlocale(locale.LC_ALL, 'de_DE.UTF-8')
try:
    locale.setlocale(locale.LC_ALL, 'de_DE.UTF-8')
except Exception as e:
    print(f"Locale setting failed: {e}")

# Define the SQL query
query = """
SELECT 
    r.name AS region, 
    d.detail AS erwerbsstatus,
    g.geschlecht,
    SUM(f.kennzahlenwert) AS summe
FROM 
    fct_zensus f
JOIN 
    dim_regionen r ON f.region_id = r.region_id
JOIN 
    dim_entitaeten e ON f.entitaet_id = e.entitaet_id AND e.entitaet_id = 5
JOIN 
    dim_geschlecht g ON f.geschlecht_id = g.geschlecht_id
JOIN 
    dim_attribute_details d ON f.attribut_id = d.attribut_id
WHERE 
    r.regionalebene = 'Land'  -- Focuses the query on the state level
GROUP BY 
    r.name, d.detail, g.geschlecht
ORDER BY 
    r.name, d.detail, g.geschlecht;
"""

# Daten laden
try:
    data = pd.read_sql(query, engine)
    print(data.head())
except Exception as e:
    print(f"Fehler beim Laden der Daten: {e}")

# Check if data is empty
if not data.empty:
    # Pivotiere die Daten für das Stacked Bar Chart
    pivot_data = data.pivot_table(index='region', columns=['erwerbsstatus', 'geschlecht'], values='summe', aggfunc='sum')
    pivot_data.fillna(0, inplace=True)

    # Erstelle ein Balkendiagramm
    fig, ax = plt.subplots(figsize=(16, 9))
    pivot_data.plot(kind='bar', stacked=True, ax=ax, colormap='viridis')

    # Beschriftungen und Titel anpassen
    ax.set_xlabel('Bundesländer', fontsize=12)
    ax.set_ylabel('Anzahl Erwerbstätige', fontsize=12)
    ax.set_title('Erwerbsstatus nach Regionalebene "Land"', fontsize=14)
    ax.tick_params(axis='x', rotation=45)
    ax.legend(title='Erwerbsstatus', fontsize=10, loc='upper left')

    # Linien an den Außenrändern entfernen
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Y-Achse dynamisch skalieren
    max_y_value = pivot_data.max().max()
    ax.set_ylim(0, max_y_value * 1.1)
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

    plt.tight_layout()
    plt.show()
else:
    print("Keine Daten vorhanden, um das Diagramm zu erstellen.")